# Using the method developed in the other notebook "*Classifying sources with the WHAV\* diagram*" can we successfully classify spaxels in SAMI galaxies? 
---

In [2]:
%matplotlib widget

In [3]:
# Imports
import sys
import os 
import numpy as np
import pandas as pd
from astropy.visualization import hist

from spaxelsleuth.loaddata.lzifu import load_lzifu_galaxies
from spaxelsleuth.loaddata.sami import load_sami_galaxies
from spaxelsleuth.plotting.plottools import plot_empty_BPT_diagram
from spaxelsleuth.plotting.plottools import vmin_fn, vmax_fn, label_fn, cmap_fn
from spaxelsleuth.plotting.plottools import bpt_colours, bpt_labels, bpt_ticks
from spaxelsleuth.plotting.plottools import morph_labels, morph_ticks
from spaxelsleuth.plotting.plottools import ncomponents_labels, ncomponents_colours
from spaxelsleuth.plotting.plottools import component_labels, component_colours
from spaxelsleuth.plotting.plotgalaxies import plot2dhistcontours, plot2dscatter, plot2dcontours

import matplotlib
from matplotlib import rc, rcParams
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

rc("text", usetex=False)
rc("font",**{"family": "serif", "size": 14})
rcParams["savefig.bbox"] = "tight"
rcParams["savefig.format"] = "pdf"
plt.ion()
plt.close("all")


In [6]:
# Options
fig_path = "/priv/meggs3/u5708159/SAMI/figs/full_sample/"
savefigs = False
bin_type = "default"    # Options: "default" or "adaptive" for Voronoi binning
ncomponents = "recom"   # Options: "1" or "recom"
eline_SNR_min = 3       # Minimum S/N of emission lines to accept
plt.close("all")


In [7]:
# Load the sample
df = load_sami_galaxies(ncomponents=ncomponents,
                        bin_type=bin_type,
                        eline_SNR_min=eline_SNR_min, 
                        vgrad_cut=False,
                        correct_extinction=False,
                        sigma_gas_SNR_cut=True)

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc,

In [119]:
# Load the LZIFU galaxies
df_lzifu = load_lzifu_galaxies(ncomponents=ncomponents,
                              bin_type=bin_type,
                              eline_SNR_min=5, 
                              vgrad_cut=False,
                              correct_extinction=False,
                              sigma_gas_SNR_cut=True)        


Loading LZIFU DataFrame for all galaxies in the LZIFU subsample...


/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:853: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:376: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus

## Classify spaxels
--- 

In [120]:
################################################################################
# Testing our classifiction system
################################################################################
df_lzifu["WHAV*"] = "Unknown"  # Initialise everything to "unknown"

#///////////////////////////////////////////////////////////////////////////////
# Step 1: filter out evolved stars
cond = df_lzifu["HALPHA EW (total)"] <= 3
df_lzifu.loc[cond, "WHAV*"] = "HOLMES"
cond_remainder = df_lzifu["WHAV*"] == "Unknown"

#///////////////////////////////////////////////////////////////////////////////
# Step 2: use the N2 ratio to divide into SF, mixed and AGN/evolved stars/shocks
# Because we used the TOTAL N2 ratio in each spaxel to determine these boundaries, these categories are representative of the DOMINANT ionisation mechanism in each spaxel.
cond_SF = cond_remainder & (df_lzifu["log N2 (total)"] < -0.35)
cond_Mixing = cond_remainder & (df_lzifu["log N2 (total)"] >= -0.35) & (df_lzifu["log N2 (total)"] < -0.15)
cond_AGN = cond_remainder & (df_lzifu["log N2 (total)"] >= -0.15)

df_lzifu.loc[cond_SF, "WHAV*"] = "SF" 
df_lzifu.loc[cond_Mixing, "WHAV*"] = "Mixing"
df_lzifu.loc[cond_AGN, "WHAV*"] = "AGN/HOLMES/shocks"

#///////////////////////////////////////////////////////////////////////////////
# For convenience: mark components as possible HOLMES 
# Question: how confident can we be that these are ALWAYS HOLMES? how common are components from e.g. LLAGN?
for ii in range(3):
    cond_possible_HOLMES = cond_AGN & (df_lzifu[f"HALPHA EW (component {ii})"] < 3) & (df_lzifu[f"sigma_gas - sigma_* (component {ii})"] < 0)
    df_lzifu.loc[cond_possible_HOLMES, f"Possible HOLMES (component {ii})"] = True
    df_lzifu.loc[~cond_possible_HOLMES, f"Possible HOLMES (component {ii})"] = False
    
#///////////////////////////////////////////////////////////////////////////////
# For convenience: mark components as being kinematically disturbed (by 3sigma)
for ii in range(3):
    cond_kinematically_disturbed = df_lzifu[f"sigma_gas - sigma_* (component {ii})"] - 3 * df_lzifu[f"sigma_gas - sigma_* error (component {ii})"] > 0
    df_lzifu.loc[cond_kinematically_disturbed, f"Kinematically disturbed (component {ii})"] = True
    df_lzifu.loc[~cond_kinematically_disturbed, f"Kinematically disturbed (component {ii})"] = False
    

In [121]:
# Test 
for ii in range(3):
    assert not df_lzifu.loc[np.isnan(df_lzifu[f"sigma_gas - sigma_* (component {ii})"]), f"Kinematically disturbed (component {ii})"].any(),\
        f"There are rows where sigma_gas - sigma_* (component {ii}) == NaN but 'Kinematically disturbed (component {ii})' == True!"

for ii in range(3):
    assert not df_lzifu.loc[np.isnan(df_lzifu[f"log HALPHA EW (component {ii})"]), f"Possible HOLMES (component {ii})"].any(),\
        f"There are rows where log HALPHA EW (component {ii}) == NaN but 'Possible HOLMES (component {ii})' == True!"
    assert not df_lzifu.loc[np.isnan(df_lzifu[f"sigma_gas - sigma_* (component {ii})"]), f"Possible HOLMES (component {ii})"].any(),\
        f"There are rows where sigma_gas - sigma_* (component {ii}) == NaN but 'Possible HOLMES (component {ii})' == True!"
    assert not df_lzifu.loc[df_lzifu[f"log HALPHA EW (component {ii})"] > 3, f"Possible HOLMES (component {ii})"].any(),\
        f"There are rows where log HALPHA EW (component {ii}) > 3 but 'Possible HOLMES (component {ii})' == True!"
    
    assert df_lzifu[df_lzifu[f"Possible HOLMES (component {ii})"] & df_lzifu[f"Kinematically disturbed (component {ii})"]].shape[0] == 0,\
        f"There are rows where both 'Possible HOLMES (component {ii})' and 'Kinematically disturbed (component {ii})' are true!"

In [23]:
################################################################################
# CHECK: plot BPT, WHAN, WHAV* for each category
################################################################################
col_z = "count"

#///////////////////////////////////////////////////////////////////////////////
# BPT - based on TOTAL fluxes
for cat in ["SF", "Mixing", "AGN/HOLMES/shocks"]:
    df_cat = df_lzifu[df_lzifu["WHAV*"] == cat]
    if df_cat.shape[0] == 0:
        continue
    col_y = "log O3 (total)"
    fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)
    
    # Plot 2D histograms of the subset
    plot2dhistcontours(df_cat, col_x="log N2 (total)", col_y=col_y, col_z=col_z, log_z=True if col_z == "count" else False, vmin=1, vmax=1e3, ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df_cat, col_x="log S2 (total)", col_y=col_y, col_z=col_z, log_z=True if col_z == "count" else False, vmin=1, vmax=1e3, ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df_cat, col_x="log O1 (total)", col_y=col_y, col_z=col_z, log_z=True if col_z == "count" else False, vmin=1, vmax=1e3, ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)

    # Decorations
    axs[1].set_title(cat)
    [ax.set_ylabel("") for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
#///////////////////////////////////////////////////////////////////////////////
# WHAN - each component shown separately
col_x = "log N2"
col_y = "log HALPHA EW"
col_z = "count"

for cat in ["HOLMES", "SF", "Mixing", "AGN/HOLMES/shocks"]:
    df_cat = df_lzifu[df_lzifu["WHAV*"] == cat]
    if df_cat.shape[0] == 0:
        continue
        
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])
    
    for ii in range(3):
        if all(df_cat[f"{col_x} (component {ii})"].isna()) or all(df_cat[f"{col_y} (component {ii})"].isna()):
            continue
        plot2dhistcontours(df_cat, 
                           col_x=f"{col_x} (component {ii})", 
                           col_y=f"{col_y} (component {ii})",
                           col_z=col_z, log_z=True if col_z == "count" else False, 
                           ax=axs[ii], nbins=100, vmin=1, vmax=1e3, contours=True, colors="white", 
                           plot_colorbar=True if ii == 2 else False)

    # Decorations
    axs[1].set_title(cat)
    [ax.set_ylabel("") for ax in axs[1:]]
    [ax.set_yticklabels([]) for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
#///////////////////////////////////////////////////////////////////////////////
# WHAV* - each component shown separately
col_x = "sigma_gas - sigma_*"
col_y = "log HALPHA EW"
col_z = "count"

for cat in ["HOLMES", "SF", "Mixing", "AGN/HOLMES/shocks"]:
    df_cat = df_lzifu[df_lzifu["WHAV*"] == cat]
    if df_cat.shape[0] == 0:
        continue
        
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])
    
    for ii in range(3):
        if all(df_cat[f"{col_x} (component {ii})"].isna()) or all(df_cat[f"{col_y} (component {ii})"].isna()):
            continue
        plot2dhistcontours(df_cat, 
                           col_x=f"{col_x} (component {ii})", 
                           col_y=f"{col_y} (component {ii})",
                           col_z=col_z, log_z=True if col_z == "count" else False, 
                           ax=axs[ii], nbins=100, vmin=1, vmax=1e3, contours=True, colors="white", 
                           plot_colorbar=True if ii == 2 else False)

    # Decorations
    axs[1].set_title(cat)
    [ax.set_ylabel("") for ax in axs[1:]]
    [ax.set_yticklabels([]) for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [125]:
# ///////////////////////////////////////////////////////////////////////////////
# SF-like spaxels
#///////////////////////////////////////////////////////////////////////////////
# Wind: number of components > 1, AND EITHER delta sigma of 1 or 2 is > 0
# Note: may want to also add if ncomponents == 1 but delta_sigma >> 0. 
# How many SF (either classified via BPT or N2) spaxels are there like this, though? Just checked - only ~0.1% have dsigma > 0 by 3sigma, so probably don't worry 
cond_SF_no_wind = cond_SF & ~(df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | df_lzifu["Kinematically disturbed (component 2)"])
df_lzifu.loc[cond_SF_no_wind, "WHAV*"] = "SF + no wind"

cond_SF_wind = cond_SF & (df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | df_lzifu["Kinematically disturbed (component 2)"])
df_lzifu.loc[cond_SF_wind, "WHAV*"] = "SF + wind"

# SF + HOLMES 
cond_SF_no_wind_HOLMES = cond_SF_no_wind & (df_lzifu["Number of components"] >= 2) & (df_lzifu["Possible HOLMES (component 0)"] | df_lzifu["Possible HOLMES (component 1)"] | df_lzifu["Possible HOLMES (component 2)"])
df_lzifu.loc[cond_SF_no_wind_HOLMES, "WHAV*"] = "SF + HOLMES + no wind"

cond_SF_wind_HOLMES = cond_SF_wind & (df_lzifu["Number of components"] >= 2) & (df_lzifu["Possible HOLMES (component 0)"] | df_lzifu["Possible HOLMES (component 1)"] | df_lzifu["Possible HOLMES (component 2)"])
df_lzifu.loc[cond_SF_wind_HOLMES, "WHAV*"] = "SF + HOLMES + wind"


# Note: what to do about low-metallicity AGN? e.g., ones that are classified as ambiguous that have log N2 < -0.35 so get lumped in with SF?

#///////////////////////////////////////////////////////////////////////////////
# Mixing-like spaxels
#///////////////////////////////////////////////////////////////////////////////
# wind/no wind
# Note: <1% of composite/mixing-like spaxels have ncomponents == 1 but delta_sigma >> 0 by 3sigma
cond_Mixing_no_wind = cond_Mixing & ~(df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | df_lzifu["Kinematically disturbed (component 2)"])
df_lzifu.loc[cond_Mixing_no_wind, "WHAV*"] = "Mixing + no wind"

cond_Mixing_wind = cond_Mixing & (df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | df_lzifu["Kinematically disturbed (component 2)"])
df_lzifu.loc[cond_Mixing_wind, "WHAV*"] = "Mixing + wind"

# Mixing + HOLMES 
cond_Mixing_no_wind_HOLMES = cond_Mixing_no_wind & (df_lzifu["Number of components"] >= 2) & (df_lzifu["Possible HOLMES (component 0)"] | df_lzifu["Possible HOLMES (component 1)"] | df_lzifu["Possible HOLMES (component 2)"])
df_lzifu.loc[cond_Mixing_no_wind_HOLMES, "WHAV*"] = "Mixing + HOLMES + no wind"

# Mixing + HOLMES + wind
cond_Mixing_wind_HOLMES = cond_Mixing_wind & (df_lzifu["Number of components"] >= 2) & (df_lzifu["Possible HOLMES (component 0)"] | df_lzifu["Possible HOLMES (component 1)"] | df_lzifu["Possible HOLMES (component 2)"])
df_lzifu.loc[cond_Mixing_wind_HOLMES, "WHAV*"] = "Mixing + HOLMES + wind"

#///////////////////////////////////////////////////////////////////////////////
# AGN-like spaxels
#///////////////////////////////////////////////////////////////////////////////
# If there is 1 component and its EW is > 0, then it's an AGN. Note that Seyfert-like components have a range of EWs, so we can't really split between LLAGN and Seyferts here - really need [OIII] for that.
cond_AGN_no_wind = cond_AGN & (df_lzifu["Number of components"] == 1) & (df_lzifu["HALPHA EW (component 0)"] > 3) & ~df_lzifu["Kinematically disturbed (component 0)"]
df_lzifu.loc[cond_AGN_no_wind, "WHAV*"] = "AGN only"

# AGN + wind
cond_AGN_nowind = cond_AGN & ~(df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | (df_lzifu["Kinematically disturbed (component 2)"] ))
df_lzifu.loc[cond_AGN_nowind, "WHAV*"] = "AGN + no wind"

cond_AGN_wind = cond_AGN & (df_lzifu["Kinematically disturbed (component 0)"] | df_lzifu["Kinematically disturbed (component 1)"] | (df_lzifu["Kinematically disturbed (component 2)"] ))
df_lzifu.loc[cond_AGN_wind, "WHAV*"] = "AGN + wind"

# If there are multiple components and at least one of them is in the HOLMES regime, then classify it as HOLMES + AGN. 
cond_AGN_nowind_HOLMES = cond_AGN_nowind & (df_lzifu["Number of components"] >= 2) & (df_lzifu["Possible HOLMES (component 0)"] | df_lzifu["Possible HOLMES (component 1)"] | df_lzifu["Possible HOLMES (component 2)"])
df_lzifu.loc[cond_AGN_nowind_HOLMES, "WHAV*"] = "AGN + HOLMES + no wind"

cond_AGN_wind_HOLMES = cond_AGN_wind & (df_lzifu["Number of components"] >= 2) & (df_lzifu["Possible HOLMES (component 0)"] | df_lzifu["Possible HOLMES (component 1)"] | df_lzifu["Possible HOLMES (component 2)"])
df_lzifu.loc[cond_AGN_wind_HOLMES, "WHAV*"] = "AGN + HOLMES + wind"

#///////////////////////////////////////////////////////////////////////////////
# Numerical labels
#///////////////////////////////////////////////////////////////////////////////
ww = -1
for whav in df_lzifu["WHAV*"].unique():
    df_lzifu.loc[df_lzifu["WHAV*"] == whav, "WHAV* (numeric)"] = ww
    ww += 1


In [126]:
df_lzifu["WHAV* (numeric)"]


0        -1.0
1        -1.0
2        -1.0
3        -1.0
4        -1.0
         ... 
100509    3.0
100512    3.0
100559    3.0
100562    3.0
100708    3.0
Name: WHAV* (numeric), Length: 503874, dtype: float64

In [116]:
df_lzifu.loc[df_lzifu["WHAV*"] == "Unknown", "log N2 (total)"].unique()


array([nan])

In [129]:
################################################################################
# CHECK: plot BPT, WHAN, WHAV* for each category
################################################################################
col_z = "WHAV* (numeric)"

#///////////////////////////////////////////////////////////////////////////////
# BPT - based on TOTAL fluxes
for cat in df_lzifu["WHAV*"].unique()[1:]:
    df_cat = df_lzifu[df_lzifu["WHAV*"] == cat]
    if df_cat.shape[0] == 0:
        continue
    col_y = "log O3 (total)"
    fig, axs, cax = plot_empty_BPT_diagram(colorbar=True, nrows=1, include_Law2021=True)
    
    # Plot 2D histograms of the subset
    plot2dhistcontours(df_cat, col_x="log N2 (total)", col_y=col_y, col_z=col_z, log_z=True if col_z == "count" else False, vmin=1, vmax=1e3, ax=axs[0], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df_cat, col_x="log S2 (total)", col_y=col_y, col_z=col_z, log_z=True if col_z == "count" else False, vmin=1, vmax=1e3, ax=axs[1], nbins=100, contours=True, colors="white", plot_colorbar=False)
    plot2dhistcontours(df_cat, col_x="log O1 (total)", col_y=col_y, col_z=col_z, log_z=True if col_z == "count" else False, vmin=1, vmax=1e3, ax=axs[2], nbins=100, contours=True, colors="white", cax=cax, plot_colorbar=True)

    # Decorations
    axs[1].set_title(cat)
    [ax.set_ylabel("") for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

ValueError: cannot reshape array of size 0 into shape (100,100)

In [109]:
#///////////////////////////////////////////////////////////////////////////////
# WHAN - each component shown separately
col_x = "log N2"
col_y = "log HALPHA EW"
col_z = "count"

for cat in df_lzifu["WHAV* classification"].unique()[1:]:
    df_cat = df_lzifu[df_lzifu["WHAV* classification"] == cat]
    if df_cat.shape[0] == 0:
        continue
        
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])
    
    for ii in range(3):
        if all(df_cat[f"{col_x} (component {ii})"].isna()) or all(df_cat[f"{col_y} (component {ii})"].isna()):
            continue
        plot2dhistcontours(df_cat, 
                           col_x=f"{col_x} (component {ii})", 
                           col_y=f"{col_y} (component {ii})",
                           col_z=col_z, log_z=True if col_z == "count" else False, 
                           ax=axs[ii], nbins=100, vmin=1, vmax=1e3, contours=True, colors="white", 
                           plot_colorbar=True if ii == 2 else False)

    # Decorations
    axs[1].set_title(cat)
    [ax.set_ylabel("") for ax in axs[1:]]
    [ax.set_yticklabels([]) for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/home/u5708159/python/Modules/spaxelsleuth/plotting/plotgalaxies.py:213: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  cax = fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [110]:
#///////////////////////////////////////////////////////////////////////////////
# WHAV* - each component shown separately
col_x = "sigma_gas - sigma_*"
col_y = "log HALPHA EW"
col_z = "count"

for cat in df_lzifu["WHAV* classification"].unique()[1:]:
    df_cat = df_lzifu[df_lzifu["WHAV* classification"] == cat]
    if df_cat.shape[0] == 0:
        continue
        
    fig, axs = plt.subplots(nrows=1, ncols=3, figsize=(15, 4))
    fig.subplots_adjust(wspace=0)
    bbox = axs[-1].get_position()
    fig.add_axes([bbox.x0 + bbox.width, bbox.y0, bbox.width * 0.1, bbox.height])
    
    for ii in range(3):
        if all(df_cat[f"{col_x} (component {ii})"].isna()) or all(df_cat[f"{col_y} (component {ii})"].isna()):
            continue
        plot2dhistcontours(df_cat, 
                           col_x=f"{col_x} (component {ii})", 
                           col_y=f"{col_y} (component {ii})",
                           col_z=col_z, log_z=True if col_z == "count" else False, 
                           ax=axs[ii], nbins=100, vmin=1, vmax=1e3, contours=True, colors="white", 
                           plot_colorbar=True if ii == 2 else False)

    # Decorations
    axs[1].set_title(cat)
    [ax.set_ylabel("") for ax in axs[1:]]
    [ax.set_yticklabels([]) for ax in axs[1:]]

    # Grid on
    [ax.grid() for ax in axs]

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  if sys.path[0] == '':


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …